In [1]:
from sympy import *
from sympy.physics.hydrogen import Psi_nlm, R_nl, E_nl
from sympy.abc import r, phi,theta, n, l, m, zeta, Z

import numpy as np
import matplotlib.pyplot as plt

In [2]:
c1, c2 = symbols('c1 c2')
zeta_1, zeta_2 = symbols('zeta_1 zeta_2', positive=True)

In [3]:
chi1 = Psi_nlm(1,0,0, r, theta, phi, zeta_1) 
chi2 = Psi_nlm(1,0,0, r, theta, phi, zeta_2) 

In [4]:
# інтеграл перекриття (радіальна частина)
S12 = 4* pi * integrate(chi1 * chi2 * r**2, (r, 0, oo))
S21 = 4* pi * integrate(chi2 * chi1 * r**2, (r, 0, oo))
S11 = 4* pi * integrate(chi1 * chi1 * r**2, (r, 0, oo))
S22 = 4* pi * integrate(chi2 * chi2 * r**2, (r, 0, oo))


In [5]:
# оператор гамільтоніана h = -1/2 ∇^2 - Z/r
# для s-орбіталі лапласіан в сферичних координатах:
def h_operator(chi, zeta):
    lap = (1/r**2)*diff(r ** 2 * diff(chi, r), r)
    return -Rational(1,2)*lap - Z/r*chi

In [6]:
# матричні елементи
h_chi1 = h_operator(chi1, zeta_1)
h_chi2 = h_operator(chi2, zeta_2)

In [7]:
H11 = 4*pi * integrate(chi1*h_chi1 * r**2, (r, 0, oo))
H22 = 4*pi * integrate(chi2*h_chi2 * r**2, (r, 0, oo))
H12 = 4*pi * integrate(chi1*h_chi2 * r**2, (r, 0, oo))

H11 = simplify(H11)
H22 = simplify(H22)
H12 = simplify(H12)

In [8]:
# Двобазисні інтеграли 
H = Matrix([[H11, H12],
              [H12, H22]])
S = Matrix([[S11, S12],
              [S21, S22]])

S = S.subs([(zeta_1, 1.45), (zeta_2, 2.91), (Z, 2)]) 
H = H.subs([(zeta_1, 1.45), (zeta_2, 2.91), (Z, 2)]) 
H

Matrix([
[         -1.84875, -1.88257712098577],
[-1.88257712098577,          -1.58595]])

In [9]:
# --- Перетворення у NumPy float ---
H = np.array(H.evalf(), dtype=float)
S = np.array(S.evalf(), dtype=float)
H

array([[-1.84875   , -1.88257712],
       [-1.88257712, -1.58595   ]])

# Матриці

Треба розв'язати секулярне рівняння вигляду
$$
(H - ES)\mathbf{c} = 0.
$$

Перетворимо це рівняння до вигляду:
$$
H\mathbf{c} = ES\mathbf{c}.
$$

Помножимо його зліва на $S^{-1}$:
$$
S^{-1}H\mathbf{c} = E\mathbf{c}.
$$

Це рівняння - є рівнянням на власні значення ($E_i$) та власні вектори $\mathbf{c}_i$ матриці $S^{-1}H$.

In [10]:
# Розв'язок generalized eigenvalue problem
eigvals, eigvecs = np.linalg.eig(np.linalg.inv(S) @ H)
eigvals

array([-1.97933487,  1.03047065])

In [11]:
eigvecs

array([[-0.86647437,  0.68999286],
       [-0.49922156, -0.72381618]])

In [13]:
0.86647437 ** 2 + 0.49922156 ** 2

0.9999999998357306